<a href="https://colab.research.google.com/github/AishwaryaPonni/Madrix_text_analysis/blob/main/tamil_sentence_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
#for stopwords removal
import nltk
from nltk.corpus import stopwords 
#for regularising the input sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
#for train-test split
from sklearn.model_selection import train_test_split
#for model building and testing
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IETE text analysis /Emotion tamil dataset.csv')

In [ ]:
df['Emotion'].value_counts()

sad         468
anger       284
fear        279
happy       268
surprise    160
shame        78
disgust      57
Name: Emotion, dtype: int64

In [ ]:
dataset= df

In [ ]:
new_df=pd.DataFrame()

In [ ]:
new_df

""


In [ ]:
sad = dataset[dataset['Emotion'] == 'sad'][:265]
anger = dataset[dataset['Emotion'] == 'anger'][:265]
fear = dataset[dataset['Emotion'] == 'fear'][:265]
happy = dataset[dataset['Emotion'] == 'happy'][:265]

In [ ]:
d = {'Sentence' : np.array(happy)[:265,2], 'Emotion' : np.array(happy)[:265,3]}
happy = pd.DataFrame(d)
new_df=new_df.append(happy, ignore_index = True)
new_df['Emotion'].value_counts()

anger    265
fear     265
happy    265
sad      265
Name: Emotion, dtype: int64

In [ ]:
new_df.to_csv('/content/drive/MyDrive/IETE text analysis /Balanced_tamil_dataset.csv ')

In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/IETE text analysis /Balanced_tamil_dataset.csv ')

In [ ]:
dataset

,Unnamed: 0,Sentence,Emotion
0,0,"ஒரு பெண், ஏற்கனவே ஒரு வேலை மற்றும் அவரது சொந்த...",sad
1,1,அவர் அதைப் பற்றி மிகவும் வருத்தமாக இருந்தார்.,sad
2,2,"நான் முகாமிட்டேன், ""ஒரு சோகமான குழந்தையைப் போல...",sad
3,3,"அவர் பொருட்டு பட்டியில் சென்றார் போது, எல்லி த...",sad
4,4,எழுத்தாளர் இப்போது நினைவு கூர்ந்தார் என்று நான...,sad
...,...,...,...
1055,1055,"இளவயது ஹீரோக்களின் இந்த புதிய மூவரும், அதன் மூ...",happy
1056,1056,டேபிள் YMor இன் மற்ற முடிவில் இருந்து லேசான கே...,happy
1057,1057,நான் யாரையும் ஏமாற்ற வெறுக்கிறேன் ஆனால் அது வழ...,happy
1058,1058,"அவர் ஒரு குறுகிய கூர்மையான பட்டை என்று, ரோஸி ஜ...",happy


In [ ]:
dataset['Emotion'].value_counts()

anger    265
fear     265
happy    265
sad      265
Name: Emotion, dtype: int64

In [ ]:
max_words = 5000
data = dataset.Sentence
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
sentences = pad_sequences(sequences)
print(sentences.shape)

(1060, 32)


In [ ]:
labels = np.array(dataset['Emotion'])
y = []
for i in range(len(labels)):
    if labels[i] == 'sad':
        y.append(0)
    if labels[i] == 'anger':
        y.append(1)
    if labels[i] == 'fear':
        y.append(2)
    if labels[i] == 'happy':
        y.append(3)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 4, dtype="float32")

In [ ]:
labels.shape

(1060, 4)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sentences,
                                                    labels,
                                                    test_size=0.15,
                                                    random_state=42)

In [ ]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    3,
        195,   42,  451,  336,  659,  621,  991,  803,   31, 4625],
      dtype=int32)

In [ ]:
print (len(X_train),len(X_test),len(y_train),len(y_test))

901 159 901 159


In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20)) #The embedding layer
model1.add(layers.LSTM(15,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(4,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

# checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='max', save_freq=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=15,validation_data=(X_test, y_test)) #,callbacks=[checkpoint1])

Epoch 1/15
29/29 [==============================] - 2s 19ms/step - loss: 1.3866 - accuracy: 0.2651 - val_loss: 1.3842 - val_accuracy: 0.3333
Epoch 2/15
29/29 [==============================] - 0s 7ms/step - loss: 1.3825 - accuracy: 0.3214 - val_loss: 1.3836 - val_accuracy: 0.3082
Epoch 3/15
29/29 [==============================] - 0s 7ms/step - loss: 1.3755 - accuracy: 0.4261 - val_loss: 1.3789 - val_accuracy: 0.3585
Epoch 4/15
29/29 [==============================] - 0s 6ms/step - loss: 1.3593 - accuracy: 0.4317 - val_loss: 1.3801 - val_accuracy: 0.2642
Epoch 5/15
29/29 [==============================] - 0s 7ms/step - loss: 1.3319 - accuracy: 0.4805 - val_loss: 1.3665 - val_accuracy: 0.3522
Epoch 6/15
29/29 [==============================] - 0s 7ms/step - loss: 1.2961 - accuracy: 0.5038 - val_loss: 1.3547 - val_accuracy: 0.3333
Epoch 7/15
29/29 [==============================] - 0s 7ms/step - loss: 1.2345 - accuracy: 0.5223 - val_loss: 1.3453 - val_accuracy: 0.3396
Epoch 8/15
29/29 [=

In [ ]:
def prediction(text):
  emotion = ['sad','anger','fear','happy']
  # l = list(text)
  sequences = tokenizer.texts_to_sequences([text])
  test = pad_sequences(sequences)
  return emotion[np.around(model1.predict(test), decimals=0).argmax(axis=1)[0]]

In [ ]:
prediction('ஒரு நாள் யாரும் அவரது டிரெய்லரில் இருந்து அவரை எழுப்ப முடியாது போது, அது லீ மோவின் அமைதியாக உள்ளிட்டு ஒரு sombre, தனியாக, சோகமான மனிதன் கண்டுபிடிக்கப்பட்டது. ')

'sad'

In [ ]:
prediction('ஆப்பிள்கள் போன்ற அன்றாட உணவுகள் ஒரு கட்டமைப்பை ஏற்படுத்தும் என்று கண்டறியும் போது தாய்மார்கள் கோபப்படுகிறார்கள் ஏன் என்று நீங்கள் காணலாம் ')

'sad'